In [1]:
import torch
from sqrll.sqrllm import SqrLLM

model_file = '../../sqrll/example/models/model2048bpe.pt'

model = SqrLLM.load(model_file).eval()

params = sum(p.numel() for p in model.parameters())
print(f'{params=:,}')

params=11,698,432


In [2]:
from tokenizers import Tokenizer

token_file = '../../sqrll/example/models/tokenizer2048.json'

tokenizer = Tokenizer.from_file(token_file)
n_vocab = tokenizer.get_vocab_size()
print(f'{n_vocab=}')

n_vocab=2048


In [3]:
from torch2cpp import codegen
from sqrll import sqrll, sqrllm

inputs = torch.tensor([[0]])
_, mem = model(inputs)
mem = [torch.zeros_like(m) for m in mem]

with open('build/model.cpp', 'w') as out_file:
    codegen(
        model,
        out_file,
        args=[inputs, mem],
        tokenizer=tokenizer,
        autowrap_functions=[
            sqrll.sqrll_kernel,
            sqrllm.rms_norm,
        ],
        skip_weights=False,
    )